In [18]:
from app.loader import add_docs_to_knowledge_base
from app.db import create_vectors_index, count_vectors
from IPython import display

In [2]:
await create_vectors_index()

Index idx:vectors created successfully


In [3]:
await add_docs_to_knowledge_base()

Loading documents


100%|██████████| 1/1 [00:04<00:00,  4.14s/it]


Loaded 1 PDF documents

Splitting documents into chunks
Inception.pdf: 127 chunks

Total chunks: 127
Min chunk size: 351 tokens
Max chunk size: 512 tokens
Average chunk size: 496 tokens

Embedding chunks


100%|██████████| 127/127 [00:02<00:00, 45.94it/s]


Adding chunks to vector DB


In [4]:
total_vectors = await count_vectors()
print(f'Total vectors: {total_vectors}')

Total vectors: 127


In [1]:
import json
from time import time
from uuid import uuid4
from IPython.display import display, JSON
from redis.asyncio import Redis
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from redis.commands.json.path import Path
from app.config import settings
from app.db import create_chat_index, create_chat, add_chat_messages, get_chat_messages, get_all_chats, delete_chats, get_chat

r = Redis(host=settings.REDIS_HOST, port=settings.REDIS_PORT)

In [3]:
await create_chat_index()

Error creating index idx:chat: Index already exists


In [35]:
res = await r.ft('idx:chat').search(Query('*').return_field("$.id", as_field="chat_id"))
print([doc.chat_id for doc in res.docs])
# await delete_chats(*[doc.chat_id for doc in res.docs])

[]


In [12]:
await add_chat_messages('1931cbac', [
    {'role': 'user', 'content': 'Hello there'},
    {'role': 'assistant', 'content': 'Hello, how can I help you?'}
])

In [12]:
chats = await get_all_chats()
print(json.dumps(chats, indent=2))

In [15]:
chat = await get_chat('chat:31c1e6d3')
print(json.dumps(chat, indent=2))

{
  "id": "31c1e6d3",
  "created": 1723207233,
  "messages": [
    {
      "role": "user",
      "content": "What is the most resilient parasite?",
      "created": 1723221771
    },
    {
      "role": "assistant",
      "content": "The most resilient parasite is an idea. As Cobb explains, \"Resilient, highly contagious. Once an idea\u2019s taken hold in the brain it\u2019s almost impossible to eradicate.\" He elaborates that while information can be forgotten, a fully formed and understood idea \"sticks\" in the mind, making it very difficult to remove.",
      "tool_calls": [
        {
          "name": "QueryKnowledgeBaseTool",
          "arguments": "{\"query_input\":\"What is the most resilient parasite?\"}"
        }
      ],
      "created": 1723221774
    },
    {
      "role": "user",
      "content": "Who is Cobb?",
      "created": 1723222424
    },
    {
      "role": "assistant",
      "content": "Cobb is a character who plays a central role in the film \"Inception.\" He 